In [1]:
import numpy

import scipy
import scipy.spatial.distance

import sys


from numba import jit

import theano

try:
    from time import perf_counter
except:
    from time import time
    perf_counter = time


INF = float('inf')


# @autojit
def closest_medoids_cost(distances, medoid_ids):
    """
    Computing the closest medoids and the cost of the new configuration
    using numpy
    """
    closest_ids = numpy.argmin(distances[:, medoid_ids], axis=1)
    return closest_ids, numpy.sum(distances[:, closest_ids])


@jit
def closest_medoids_numba(distances, medoid_ids, clustering):
    """
    Computing the closest medoids and the cost of the new configuration
    using a simple loop optimized with Numba
    """
    n_instances = distances.shape[0]
    tot_cost = 0
    for i in range(n_instances):
        best_medoid = i
        best_cost = INF
        for j in medoid_ids:
            cost = distances[i, j]
            if cost < best_cost:
                best_cost = cost
                best_medoid = j

        tot_cost += best_cost
        clustering[i] = best_medoid

    return tot_cost


def create_theano_cost_function(distances):
    """
    Creating the function to compute the closest medoids and the cost
    of that configuration using Theano
    """
    #
    # the distance matrix is a shared var
    #
    D = theano.shared(distances)
    M = theano.tensor.bvector()
    #
    # Some tricky stuff here, I am using max_and_argmax to get both
    # things so I shall use an inverted distance matrix
    #
    Inf = theano.shared(float('inf'))
    D_meds = theano.tensor.switch(M, -D, -Inf)
    # D_meds = theano.printing.Print('DM')(D_meds_a)
    Closest, Idx = theano.tensor.max_and_argmax(D_meds, axis=1)
    # Closest = theano.printing.Print('CLS')(Closest_a)
    # Idx = theano.printing.Print('IDX')(Idx_a)
    Cost = theano.tensor.sum(Closest)
    return theano.function([M], [-Cost, Idx])


def medoid_assoc_to_clustering(clustering):
    """
    Converting a clustering by medoid ids into one with sequential ids
    (starting from 0)
    """
    medoids_to_clusters = set(clustering)

    clustering_assoc = {medoid_id: cluster_id for cluster_id, medoid_id
                        in enumerate(medoids_to_clusters)}
    clustering_ids = [clustering_assoc[medoid_id] for medoid_id in clustering]
    return clustering_ids

# @autojit


def pam(distances,
        k,
        n_iters=100,
        delta_cost=1e-5,
        rand_gen=None,
        medoids_2_clusters=False,
        theano=False):
    """
    Partitioning Around Medoids (PAM) implementation of K-Medoids
    """
    n_instances = distances.shape[0]
    stop = False
    iter = 0

    if theano:
        closest_medoids_theano = create_theano_cost_function(distances)

    if rand_gen is None:
        rand_gen = numpy.random.RandomState(1337)

    instance_ids = [i for i in range(n_instances)]
    #
    # using an array of bits
    medoid_ids_vec = numpy.zeros(n_instances, dtype=bool)

    # setting k random medoids
    medoid_ids = rand_gen.choice(instance_ids, k, replace=False)
    # medoid_ids = [1, 7]
    medoid_ids_vec[medoid_ids] = True

    #
    # using another array for the medoid correspondences
    # assigning to the closest medoid
    if not theano:
        clustering = numpy.zeros(n_instances, dtype='int32')
        cost = closest_medoids_numba(distances, medoid_ids, clustering)

    else:
        cost, clustering = closest_medoids_theano(medoid_ids_vec)

    print('Initial cost {0} clustering {1} medoids {2} vec {3}'.
          format(cost, clustering, medoid_ids, medoid_ids_vec))

    best_cost = cost
    best_clustering = clustering
    curr_best_cost = cost
    curr_best_clustering = numpy.copy(clustering)
    curr_best_medoid_ids = medoid_ids
    curr_best_medoid_ids_vec = medoid_ids_vec

    while iter < n_iters and not stop:

        iter_start_t = perf_counter()
        #
        # Maximization step
        #
        for i, medoid_id in enumerate(medoid_ids):

            avg_swap_t = 0
            #
            # Trying to swap each other instance
            #
            for j in instance_ids:

                swap_start_t = perf_counter()

                # j is not a medoid
                if not medoid_ids_vec[j]:

                    # remove medoid
                    # medoid_ids_cp = set(medoid_ids)
                    # medoid_ids_cp.remove(medoid_id)
                    # add instance
                    # medoid_ids_cp.add(j)

                    # print('\n curr vec', medoid_ids_vec)
                    medoid_ids_vec[medoid_id] = False
                    medoid_ids_vec[j] = True

                    medoid_ids_cp, = numpy.where(medoid_ids_vec)

                    # compute the Expected cost
                    # clustering, cost = closest_medoids_cost(distances,
                    #                                         medoid_ids_vec)
                    if not theano:
                        cost = closest_medoids_numba(distances,
                                                     medoid_ids_cp,
                                                     clustering)
                    else:
                        cost, clustering = closest_medoids_theano(
                            medoid_ids_vec)

                    #       format(cost, clustering,
                    #              medoid_ids_cp, medoid_ids_vec))

                    if cost < curr_best_cost:
                        print(
                            '\nnew best cost {0}/{1}'.format(cost,
                                                             curr_best_cost))
                        curr_best_cost = cost
                        curr_best_clustering = numpy.copy(clustering)
                        curr_best_medoid_ids_vec = numpy.copy(medoid_ids_vec)
                        curr_best_medoid_ids = medoid_ids_cp

                    medoid_ids_vec[medoid_id] = True
                    medoid_ids_vec[j] = False

                swap_end_t = perf_counter()
                avg_swap_t += (swap_end_t - swap_start_t)
                sys.stdout.write('\rswapped {0}-{1} [{2:.4f} secs avg]'.
                                 format(i, j, avg_swap_t / (j + 1)))
                sys.stdout.flush()

        #
        # checking for the cost as well?
        #
        delta_cost = best_cost - curr_best_cost

        #
        # checking for changes in the clustering scheme
        #
        if numpy.any(medoid_ids_vec != curr_best_medoid_ids_vec):
            print('\nNew clustering', best_clustering)
            best_clustering = curr_best_clustering
            medoid_ids = curr_best_medoid_ids
            medoid_ids_vec = curr_best_medoid_ids_vec
            best_cost = curr_best_cost
        else:
            stop = True

        iter_end_t = perf_counter()
        print('\n-->Elapsed {0:.4f} secs for iteration {1}/{2} COST:{3}'.
              format(iter_end_t - iter_start_t,
                     iter + 1,
                     n_iters,
                     best_cost))
        iter += 1

    #
    # translating from medoids to clusters?
    #
    print('Best clustering (medoid ids)', best_clustering)
    if medoids_2_clusters:
        best_clustering = medoid_assoc_to_clustering(best_clustering)
        print('Best clustering (cluster ids)', best_clustering)

    return best_clustering

def compute_similarity_matrix(data_slice):
    """
    From a matrix m x n creates a kernel matrix
    according to a metric of size m x m
    (it shall be symmetric, and (semidefinite) positive)
    ** USES SCIPY **
    """

    pairwise_dists = \
        scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(data_slice,
                                         'sqeuclidean'))

    return pairwise_dists


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\MissBleu\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [10]:

d = numpy.array([[0, 3, 3, 3, 8, 6, 8, 7, 7, 5],
                             [3, 0, 4, 4, 5, 3, 4, 4, 6, 6],
                             [3, 4, 0, 2, 9, 7, 9, 8, 8, 6],
                             [3, 4, 2, 0, 7, 5, 7, 6, 6, 4],
                             [8, 5, 9, 7, 0, 2, 2, 3, 5, 5],
                             [6, 3, 7, 5, 2, 0, 2, 1, 3, 3],
                             [8, 4, 9, 7, 2, 2, 0, 1, 3, 3],
                             [7, 4, 8, 6, 3, 1, 1, 0, 2, 2],
                             [7, 6, 8, 6, 5, 3, 3, 2, 0, 2],
                             [5, 6, 6, 4, 5, 3, 3, 2, 2, 0]])

distances = compute_similarity_matrix(d)
k = 2

clustering = pam(distances, k, n_iters=20, theano=False)


Initial cost 268.0 clustering [3 3 3 3 6 6 6 6 6 6] medoids [6 3] vec [False False False  True False False  True False False False]
swapped 0-6 [0.0113 secs avg]
new best cost 240.0/268.0
swapped 1-9 [0.0000 secs avg]
New clustering [9 9 9 9 6 6 6 6 9 9]

-->Elapsed 0.0963 secs for iteration 1/20 COST:240.0

new best cost 238.0/240.0
swapped 1-9 [0.0000 secs avg]
New clustering [3 3 3 3 7 7 7 7 7 7]

-->Elapsed 0.0280 secs for iteration 2/20 COST:238.0
swapped 1-9 [0.0000 secs avg]
-->Elapsed 0.0269 secs for iteration 3/20 COST:238.0
Best clustering (medoid ids) [0 0 0 0 7 7 7 7 7 7]


In [9]:
print('Creating synthetic data')
synth_data = numpy.random.binomial(100, 0.5, (2000, 15))
synth_data

Creating synthetic data


(2000, 15)

In [71]:
import pandas as pd
data = pd.read_csv("albumtsne.csv")

print("Success!")


data.info()

Success!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172772 entries, 0 to 172771
Data columns (total 7 columns):
Unnamed: 0    172772 non-null int64
albums        172772 non-null int64
all_genres    172772 non-null float64
avg           33898 non-null float64
styles        33307 non-null float64
country       34194 non-null float64
label         34203 non-null float64
dtypes: float64(5), int64(2)
memory usage: 9.2 MB


In [12]:
X = data.iloc[:,1:7]

print("ready")

X.head()

ready


,albums,all_genres,avg,styles,country,label
0,10,6.0,3.444444,8.0,1.0,1.0
1,8,1.0,2.571429,6.0,3.0,5.0
2,2,1.0,0.000000,1.0,1.0,1.0
3,41,1.0,0.600000,27.0,5.0,18.0
4,41,1.0,NaN,NaN,NaN,NaN


In [15]:
cols = ["albums", "all_genres", "avg", "styles", "country", "label"]
for i in cols:
    a = X[i].mean()
    X.loc[X[i].isna(), i] = a
X.head(100)

,albums,all_genres,avg,styles,country,label
0,10.0,6.0,3.444444,8.000000,1.000000,1.00000
1,8.0,1.0,2.571429,6.000000,3.000000,5.00000
2,2.0,1.0,0.000000,1.000000,1.000000,1.00000
3,41.0,1.0,0.600000,27.000000,5.000000,18.00000
4,41.0,1.0,2.040990,3.728015,1.987074,3.50671
5,11.0,2.0,0.525000,9.000000,5.000000,14.00000
6,26.0,1.0,2.040990,3.728015,1.987074,3.50671
7,3.0,6.0,0.400000,8.000000,2.000000,2.00000
8,3.0,1.0,0.960000,8.000000,3.000000,15.00000
9,4.0,1.0,0.000000,1.000000,2.000000,1.00000


In [16]:

if __name__ == '__main__':

    #
    # create sample dataset
    #
    print('Creating sampled data')
    sample_data = X.sample(500).values


    #
    # create distance matrix
    #
    print('Computing the distance matrix')
    distances = compute_similarity_matrix(sample_data)

    #
    # clustering
    #
    k = 10
clustering = pam(distances, k, theano=False)

Creating sampled data
Computing the distance matrix
Initial cost 15649.690912213347 clustering [280 197 124 280 280 446 280 280 446 280 280 280 446 139 446 124 446 197
 280 197 280 144 280 197 197 280 124 197 280 144 446 124 144 280 124 280
 124 144 124 197 124 124 197 139 197 280 280 124 280 124 280 446 446 124
 197 446 197 124 280 144 446 124 280 280 280 446 280 280 197 124 197 197
 144 197 280 280 280 280 446 197 124 280 197 280 124 280 124 197 280 124
 197 197 197 197 446 197 197 124 197 124 197 280 280 124 280 446 197 280
 280 446 139 280 446 280 280 197 197 280 197 124 197 280 197 446 124 124
 124 280 446 446 280 124 446 280 197 280 280 280 280 139 124 197 139 280
 144 197 124 446 280 280 124 280 446 280 280 280 280 124 280 197 446 280
 124 280 124 280 124 446 280 139 197 280 280 280 144 197 124 280 197 446
 197 124 280 280 144 280 124 280 197 124 280 280 144 280 124 197 280 197
 124 446 280 280 280 124 446 139 280 280 124 197 124 124 280 197 197 197
 197 280 139 280 124 139 197 

swapped 1-36 [0.0002 secs avg]
new best cost 5550.681828157597/6026.41724166758
swapped 1-122 [0.0002 secs avg]
new best cost 5254.419859968216/5550.681828157597
swapped 1-282 [0.0001 secs avg]
new best cost 5194.6066808172145/5254.419859968216
swapped 1-446 [0.0001 secs avg]
new best cost 5106.809916166908/5194.6066808172145
swapped 2-122 [0.0001 secs avg]
new best cost 4835.043944714531/5106.809916166908
swapped 2-282 [0.0001 secs avg]
new best cost 4775.23076556353/4835.043944714531
swapped 2-446 [0.0001 secs avg]
new best cost 4687.434000913224/4775.23076556353
swapped 9-499 [0.0001 secs avg]
New clustering [127 197 124 127 127  30 127 127 446 127 127 127 446 139  30 124 446 197
 127 197 127 144 127 197 197 127 124 197 127 144  30 124 144 127 124 127
 124 144 124 197 124 124 197 139 197 127 127 124 127 124 127 446  30 124
 197  30 197 124 127 144 446 124 127 127 127 446 127 127 197 124 197 197
 144 197 127 127 127 127 446 197 124 127 197 127 124 127 124 197 127 124
 197 197 197 197

swapped 3-12 [0.0001 secs avg]]
new best cost 2925.24559495264/3051.1076900096996
swapped 3-24 [0.0001 secs avg]
new best cost 2915.762709984695/2925.24559495264
swapped 3-36 [0.0001 secs avg]
new best cost 2909.8831321526504/2915.762709984695
swapped 3-42 [0.0001 secs avg]
new best cost 2904.148977738487/2909.8831321526504
swapped 3-106 [0.0001 secs avg]
new best cost 2886.558147037479/2904.148977738487
swapped 9-499 [0.0001 secs avg]
New clustering [366 197 124 366 366  30 366 366 258 366 366 366 446 139  30 124 258 197
 366 197 366 197 366 197 197 366 124 197 366 197  30 124 139 366 124 366
 124 139 124 197 124 124 197 139 197 366 366 124 366 124 366 446 109 124
 197 109 197 124 366 197 446 124 366 366 366 258 366 366 197 124 197 197
 197 197 366 366 366 366 446 197 124 366 197 366 124 366 124 197 366 124
 197 197 197 197 258 197 197 124 197 124 197 366 366 124 366 258 197 366
 366 109 139 366 446 366 366 197 197 366 197 124 197 366 197 447 124 124
 124 366 446 258 366 124 258 366 1

In [22]:
if __name__ == '__main__':

    #
    # create sample dataset
    #
    print('Creating sampled data')
    sample_data = X.sample(5000).values


    #
    # create distance matrix
    #
    print('Computing the distance matrix')
    distances = compute_similarity_matrix(sample_data)

    #
    # clustering
    #
    k = 7
clustering = pam(distances, k, theano=False)

Creating sampled data
Computing the distance matrix
Initial cost 119894.90253893522 clustering [3493 2319 3226 ..., 3434 2319 2319] medoids [3226 4015 3434 4169 3541 2319 3493] vec [False False False ..., False False False]

new best cost 102438.90253893516/119894.90253893522
swapped 0-38 [0.0010 secs avg]
new best cost 98457.90253893514/102438.90253893516
swapped 0-65 [0.0010 secs avg]
new best cost 81844.76889625825/98457.90253893514
swapped 0-125 [0.0010 secs avg]
new best cost 80153.90253893526/81844.76889625825
swapped 0-156 [0.0010 secs avg]
new best cost 78470.90253893525/80153.90253893526
swapped 0-1061 [0.0010 secs avg]
new best cost 78430.90253893525/78470.90253893525
swapped 0-2661 [0.0010 secs avg]
new best cost 78322.90253893523/78430.90253893525
swapped 1-125 [0.0010 secs avg]]
new best cost 77307.90253893523/78322.90253893523
swapped 1-156 [0.0010 secs avg]
new best cost 75624.90253893522/77307.90253893523
swapped 1-1061 [0.0010 secs avg]
new best cost 75584.90253893522/

In [23]:
seven = pd.value_counts(pd.Series(clustering))
seven

4169    2795
2319    1169
842      468
327      282
244      138
2002      92
4590      56
dtype: int64

In [21]:
ten = pd.value_counts(pd.Series(clustering))
ten

197    328
49      75
375     34
258     25
497     12
37      10
109      7
447      4
434      3
321      2
dtype: int64

In [24]:
if __name__ == '__main__':

    #
    # create sample dataset
    #
    print('Creating sampled data')
    sample_data = X.sample(10000).values


    #
    # create distance matrix
    #
    print('Computing the distance matrix')
    distances = compute_similarity_matrix(sample_data)

    #
    # clustering
    #
    k = 6
clustering = pam(distances, k, theano=False)

Creating sampled data
Computing the distance matrix
Initial cost 438258.17633012537 clustering [9927 9927 7707 ..., 9927 7707 6206] medoids [9927 3245 2303 7707 8820 6206] vec [False False False ..., False False False]

new best cost 313233.16728354647/438258.17633012537
swapped 0-0 [0.0013 secs avg]
new best cost 312881.1206394291/313233.16728354647
swapped 0-39 [0.0021 secs avg]
new best cost 293460.17633012636/312881.1206394291
swapped 0-46 [0.0021 secs avg]
new best cost 282636.4634551984/293460.17633012636
swapped 0-60 [0.0021 secs avg]
new best cost 276966.17633012665/282636.4634551984
swapped 0-63 [0.0021 secs avg]
new best cost 185610.17633012668/276966.17633012665
swapped 0-208 [0.0019 secs avg]
new best cost 184741.17633012668/185610.17633012668
swapped 0-254 [0.0019 secs avg]
new best cost 183962.1763301267/184741.17633012668
swapped 0-2655 [0.0018 secs avg]
new best cost 183957.17633012668/183962.1763301267
swapped 0-3471 [0.0018 secs avg]
new best cost 183605.1763301267/18

In [26]:
import pandas as pd
sixClusters = pd.DataFrame(sample_data)

sixClusters.columns = ["albums", "all_genres", "avg", "styles", "country", "label"]


In [28]:
sixClusters["cluster"] = clustering
sixClusters.to_csv("sixclusters.csv")

In [30]:
ten = pd.value_counts(pd.Series(clustering))
ten

8820    5484
44      2920
917      948
755      342
4263     164
7483     142
dtype: int64

In [29]:
sixClusters.head(100)

,albums,all_genres,avg,styles,country,label,cluster
0,12.0,2.0,1.625000,11.000000,3.000000,7.00000,917
1,10.0,1.0,0.000000,3.000000,1.000000,1.00000,917
2,2.0,1.0,2.040990,3.728015,1.987074,3.50671,8820
3,3.0,2.0,2.040990,3.728015,1.987074,3.50671,8820
4,2.0,1.0,2.040990,3.728015,1.987074,3.50671,8820
5,5.0,1.0,1.333333,2.000000,2.000000,3.00000,44
6,3.0,1.0,2.040990,3.728015,1.987074,3.50671,8820
7,6.0,1.0,2.040990,3.728015,1.987074,3.50671,44
8,5.0,1.0,2.040990,3.728015,1.987074,3.50671,44
9,2.0,1.0,2.040990,3.728015,1.987074,3.50671,8820


In [32]:
ls = list(sixClusters.cluster.unique())
ls

[917, 8820, 44, 7483, 755, 4263]

In [37]:
first = pd.DataFrame(sixClusters[(sixClusters["cluster"] == 44)])
second = pd.DataFrame(sixClusters[(sixClusters["cluster"] == 755)])
third = pd.DataFrame(sixClusters[(sixClusters["cluster"] == 917)])
fourth = pd.DataFrame(sixClusters[(sixClusters["cluster"] == 4263)])
fifth = pd.DataFrame(sixClusters[(sixClusters["cluster"] == 7483)])
sixth = pd.DataFrame(sixClusters[(sixClusters["cluster"] == 8820)])

In [72]:
ranges = {}
def findrange(df, i):
    for i in df.columns:
        
        ranges[i]=([df[i].max(), df[i].min(), df[i].mean()])
    return ranges


    
    

In [73]:
findrange(first, 5)

{'albums': [8.0, 2.0, 5.250684931506849],
 'all_genres': [15.0, 1.0, 1.6791095890410959],
 'avg': [29.0, 0.0, 2.0866135101713357],
 'cluster': [44, 44, 44.0],
 'country': [6.0, 1.0, 1.9347566383912373],
 'label': [13.0, 1.0, 3.3400064341868405],
 'styles': [13.0, 1.0, 3.555658473618839]}

In [74]:
findrange(second, 5)

{'albums': [29.0, 18.0, 22.698830409356724],
 'all_genres': [11.0, 1.0, 2.3157894736842106],
 'avg': [25.0, 0.0, 2.095917749900145],
 'cluster': [755, 755, 755.0],
 'country': [8.0, 1.0, 1.9668190137663153],
 'label': [17.0, 1.0, 3.5036195142418367],
 'styles': [15.0, 1.0, 3.706842001672553]}

In [75]:
findrange(third, 5)

{'albums': [17.0, 9.0, 11.891350210970463],
 'all_genres': [12.0, 1.0, 1.878691983122363],
 'avg': [20.0, 0.0, 2.016060853496144],
 'cluster': [917, 917, 917.0],
 'country': [6.0, 1.0, 1.9226584154102984],
 'label': [25.0, 1.0, 3.429681292299089],
 'styles': [16.0, 1.0, 3.6236993680703673]}

In [76]:
findrange(fourth, 5)

{'albums': [49.0, 30.0, 36.55487804878049],
 'all_genres': [9.0, 1.0, 2.9878048780487805],
 'avg': [4.0, 0.0, 1.8458146187832942],
 'cluster': [4263, 4263, 4263.0],
 'country': [8.0, 1.0, 1.9725643637380375],
 'label': [20.0, 1.0, 3.657185077902871],
 'styles': [38.0, 1.0, 3.9705961612112652]}

In [77]:
findrange(fifth, 5)

{'albums': [15.0, 2.0, 4.028169014084507],
 'all_genres': [40.0, 1.0, 8.028169014084508],
 'avg': [3.583333333333333, 0.27906976744186052, 0.993974298884503],
 'cluster': [7483, 7483, 7483.0],
 'country': [12.0, 1.0, 4.852112676056338],
 'label': [30.0, 2.0, 11.711267605633802],
 'styles': [42.0, 5.0, 14.119718309859154]}

In [78]:
findrange(sixth, 5)

{'albums': [3.0, 2.0, 2.313822027716995],
 'all_genres': [5.0, 1.0, 1.1312910284463895],
 'avg': [37.0, 0.0, 2.056630295527933],
 'cluster': [8820, 8820, 8820.0],
 'country': [8.0, 1.0, 1.9296363304297521],
 'label': [14.0, 1.0, 3.3873232004999934],
 'styles': [13.0, 1.0, 3.525314095509436]}

In [79]:
ten = pd.value_counts(pd.Series(clustering))
ten

8820    5484
44      2920
917      948
755      342
4263     164
7483     142
dtype: int64